# File Interaction and Q&A:
This agent is designed to interact with a local file system to perform three primary tasks:

1. List files in a specified directory
2. Read file contents
3. Answer user questions based on the content of those files

# The agent loop 
The Agent Loop is the core engine driving an AI agent’s decision-making process.

It allows the agent to interact dynamically with its environment by continuously generating actions, executing them, and adapting based on the results.

This loop makes the agent capable of completing complex tasks step by step.

In [1]:
import os
from dotenv import load_dotenv

# Get API key from environment variables
load_dotenv()
api_key = os.environ.get('OPENAI_API_KEY')

if api_key:
    print("API key loaded successfully!")
else:
    print("Error: API key not found in .env file")


API key loaded successfully!


In [4]:
from litellm import completion
from typing import List, Dict
import sys

def generate_response(messages: List[Dict]) -> str:
    """Call LLM to get response"""
    response = completion(
      model="openai/gpt-4o",
      messages=messages,
      max_tokens=1000
   )
    return response.choices[0].message.content


In [5]:
def parse_action(response):
    """Parse the agent's response to extract the action and its parameters."""
    try:
        # Extract the action JSON from between ```action and ``` tags
        action_text = response.split("```action")[1].split("```")[0].strip()
        action = json.loads(action_text)
        return action
    except Exception as e:
        print(f"Error parsing action: {e}")
        return {
            "tool_name": "error",
            "args": {"message": "Failed to parse the action. Make sure the response follows the correct format."}
        }

In [6]:
def list_files():
    """List all files in the current directory."""
    try:
        files = [f for f in os.listdir('.') if os.path.isfile(f)]
        return files
    except Exception as e:
        return f"Error listing files: {e}"

In [7]:
def read_file(file_name):
    """Read the content of a file."""
    try:
        if not os.path.exists(file_name):
            return f"Error: File '{file_name}' does not exist."
        
        with open(file_name, 'r', encoding='utf-8') as file:
            content = file.read()
        return content
    except Exception as e:
        return f"Error reading file: {e}"